## Differences between sequential and functional API models in ®Keras

### This notebooks present the differences in number of parameters in API functional and Sequential models. 

#### As you can see below, there are differences between output shapes in each layers which provides completely different number parameters and learning time / accuracy.

This is next step of my investigation about this topic. I discovered it before, however now I add generator here to dismiss any additional features which might cause that issue. The next step is to create equivalent of such a model (even without training) in Keras and try to understand what's *under the hood* of this problem.

In [1]:
import tensorflow as tf
from tensorflow.python.client import device_lib
sess = tf.Session(config=tf.ConfigProto(log_device_placement=True))
device_lib.list_local_devices()

[name: "/cpu:0"
 device_type: "CPU"
 memory_limit: 268435456
 locality {
 }
 incarnation: 3865598924993322535]

In [2]:
import numpy as np
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential, Model, load_model
from keras.layers import Conv2D, MaxPooling2D
from keras.layers import Activation, Dropout, Flatten, Dense, Input
from keras.callbacks import ModelCheckpoint
from keras.utils.vis_utils import plot_model
from sklearn.utils import shuffle
from keras import backend as K
import matplotlib.pyplot as plt

Using TensorFlow backend.


In [3]:
width, height = 640, 480
train_data_dir = '../kostki/gen_x2/train'
test_data_dir = '../kostki/gen_x2/test'
nb_train_samples = 9446
nb_test_samples = 2362
epochs = 25
batch_size = 4

input_shape = (width, height, 1)

## Sequential model

In [4]:
model = Sequential()
model.add(Conv2D(64, kernel_size=(3, 3), input_shape=input_shape, activation='relu'))
model.add(Conv2D(64, kernel_size=(3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(128, (3, 3), activation='relu'))
model.add(Conv2D(128, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(256, (3, 3), activation='relu'))
model.add(Conv2D(256, (3, 3), activation='relu'))
model.add(Conv2D(256, (3, 3), activation='relu'))
model.add(Conv2D(256, (3, 3), activation='relu'))
model.add(Dropout(0.25))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(512, (3, 3), activation='relu'))
model.add(Conv2D(512, (3, 3), activation='relu'))
model.add(Conv2D(512, (3, 3), activation='relu'))
model.add(Conv2D(512, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Flatten())
model.add(Dense(2048, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(2048, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(128, activation='relu'))
model.add(Dense(6, activation='softmax'))

## Api functional model

In [5]:
visible = Input(shape=input_shape)

conv1 = Conv2D(64, kernel_size=(3, 3), padding='same', activation='relu')(visible)
conv2 = Conv2D(64, kernel_size=(3, 3), padding='same', activation='relu')(conv1)
pool1 = MaxPooling2D((2, 2))(conv2)

conv3 = Conv2D(128, kernel_size=(3, 3), padding='same', activation='relu')(pool1)
conv4 = Conv2D(128, kernel_size=(3, 3), padding='same', activation='relu')(conv3)
pool2 = MaxPooling2D((2, 2))(conv4)

conv5 = Conv2D(256, kernel_size=(3, 3), padding='same', activation='relu')(pool2)
conv6 = Conv2D(256, kernel_size=(3, 3), padding='same', activation='relu')(conv5)
conv7 = Conv2D(256, kernel_size=(3, 3), padding='same', activation='relu')(conv6)
conv8 = Conv2D(256, kernel_size=(3, 3), padding='same', activation='relu')(conv7)
dropout1 = Dropout(0.2)(conv8)
pool3 = MaxPooling2D((2, 2))(dropout1)

conv9 = Conv2D(256, kernel_size=(3, 3), padding='same', activation='relu')(pool3)
conv10 = Conv2D(256, kernel_size=(3, 3), padding='same', activation='relu')(conv9)
conv11 = Conv2D(256, kernel_size=(3, 3), padding='same', activation='relu')(conv10)
conv12 = Conv2D(256, kernel_size=(3, 3), padding='same', activation='relu')(conv11)
pool4 = MaxPooling2D((2, 2))(conv12)

flat1 = Flatten()(pool4)
hidden1 = Dense(2048, activation='relu')(flat1)
dropout2 = Dropout(0.5)(hidden1)
hidden2 = Dense(2048, activation='relu')(dropout2)
dropout3 = Dropout(0.5)(hidden2)
hidden3 = Dense(128, activation='relu')(dropout3)
predictions = Dense(6, activation='softmax')(hidden3)
modelAPI = Model(inputs=visible, outputs=predictions)
modelAPI.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [6]:
model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

In [7]:
modelAPI.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

In [8]:
# summarize model and apply checkpoints
print(model.summary())
filepath = 'generator2-{epoch:02d}-{loss:.4f}.h5'
checkpoint = ModelCheckpoint(filepath, monitor='loss', verbose=1, save_best_only=True, mode='min')
callbacks_list = [checkpoint]

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 638, 478, 64)      640       
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 636, 476, 64)      36928     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 318, 238, 64)      0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 316, 236, 128)     73856     
_________________________________________________________________
conv2d_4 (Conv2D)            (None, 314, 234, 128)     147584    
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 157, 117, 128)     0         
_________________________________________________________________
conv2d_5 (Conv2D)            (None, 155, 115, 256)     295168    
__________

In [9]:
# summarize model and apply checkpoints
print(modelAPI.summary())
filepathAPI = 'generatorAPI-{epoch:02d}-{loss:.4f}.h5'
checkpointAPI = ModelCheckpoint(filepath, monitor='loss', verbose=1, save_best_only=True, mode='min')
callbacks_listAPI = [checkpointAPI]

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 640, 480, 1)       0         
_________________________________________________________________
conv2d_13 (Conv2D)           (None, 640, 480, 64)      640       
_________________________________________________________________
conv2d_14 (Conv2D)           (None, 640, 480, 64)      36928     
_________________________________________________________________
max_pooling2d_5 (MaxPooling2 (None, 320, 240, 64)      0         
_________________________________________________________________
conv2d_15 (Conv2D)           (None, 320, 240, 128)     73856     
_________________________________________________________________
conv2d_16 (Conv2D)           (None, 320, 240, 128)     147584    
_________________________________________________________________
max_pooling2d_6 (MaxPooling2 (None, 160, 120, 128)     0         
__________

In [10]:
train_datagen = ImageDataGenerator(
    rescale=1. / 255)
test_datagen = ImageDataGenerator(rescale=1. / 255)

In [11]:
train_generator = train_datagen.flow_from_directory(
    train_data_dir,
    target_size=(width, height),
    color_mode='grayscale',
    batch_size=batch_size,
    class_mode='categorical',
    classes=['1', '2', '3', '4', '5', '6'],
    shuffle=True,
    seed=2018)

test_generator = test_datagen.flow_from_directory(
    test_data_dir,
    target_size=(width, height),
    batch_size=batch_size,
    color_mode='grayscale',
    class_mode='categorical',
    classes=['1', '2', '3', '4', '5', '6'],
    shuffle=True,
    seed=2018)

Found 9446 images belonging to 6 classes.
Found 2362 images belonging to 6 classes.


In [ ]:
history = model.fit_generator(
    train_generator,
    steps_per_epoch=nb_train_samples // batch_size,
    epochs=epochs,
    verbose=1,
    callbacks=callbacks_list,
    shuffle=True,
    validation_data=test_generator,
    validation_steps=nb_test_samples // batch_size)

In [ ]:
historyAPI = modelAPI.fit_generator(
    train_generator,
    steps_per_epoch=nb_train_samples // batch_size,
    epochs=epochs,
    verbose=1,
    callbacks=callbacks_list,
    shuffle=True,
    validation_data=test_generator,
    validation_steps=nb_test_samples // batch_size)

In [ ]:
model_json = model.to_json()
with open('generator2_model.json', 'w') as json_file:
    json_file.write(model_json)
    
model.save_weights('generator2_weights.h5')
print('Saved model weights')

In [ ]:
modelAPI_json = modelAPI.to_json()
with open('generatorAPI_model.json', 'w') as json_file:
    json_file.write(modelAPI_json)
    
modelAPI.save_weights('generatorAPI_weights.h5')
print('Saved modelAPI weights')